In [ ]:
!pip install scikit-learn transformers lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 514.5 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/13.3 MB 239.0 kB/s eta 0:00:45

In [ ]:
# Import necessary libraries
import lime
import lime.lime_tabular
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Load the Iris dataset
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=42)

# Train a Random Forest Classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Predict on a test instance
test_instance = X_test[0]
print("Predicted class:", clf.predict(test_instance.reshape(1, -1)))

# Use LIME to explain the prediction
explainer = lime.lime_tabular.LimeTabularExplainer(X_train,
                                                   feature_names=iris.feature_names,
                                                   class_names=iris.target_names,
                                                   discretize_continuous=True)
exp = explainer.explain_instance(test_instance, clf.predict_proba, num_features=4, top_labels=1)
exp.show_in_notebook()


In [ ]:
# Import required modules
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from lime.lime_text import LimeTextExplainer

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

# Define the prediction function for LIME
def predictor(texts):
    inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1).detach().numpy()
    return probs

# Initialize LIME's text explainer
explainer = LimeTextExplainer(class_names=['negative', 'neutral', 'positive'])



In [ ]:
# Sample tweet to explain
tweet = "I love using the new feature! So helpful."

# Generate the explanation
exp = explainer.explain_instance(tweet, predictor, num_features=5, top_labels=3)

exp.show_in_notebook()


In [ ]:
# Sample tweet to explain for negative
tweet = "I hate using the new feature! So annoying."

# Generate the explanation
exp = explainer.explain_instance(tweet, predictor, num_features=5, top_labels=3)

exp.show_in_notebook()


In [ ]:
# Sample tweet to explain for neutral (bias at work)
tweet = "I am using the new feature."

# Generate the explanation
exp = explainer.explain_instance(tweet, predictor, num_features=5, top_labels=3)

exp.show_in_notebook()


In [ ]:
# Sample tweet to explain for neutral (bias at work)
tweet = "I am using the old feature."

# Generate the explanation
exp = explainer.explain_instance(tweet, predictor, num_features=5, top_labels=3)

exp.show_in_notebook()
